### Imports and setup

In [3]:
# %pip install git+https://github.com/GTSF-Quantitative-Sector/sec_parser.git python-dotenv tqdm pickle matplotlib pandas

In [4]:
from sec import constants, stock, lookups, processor
import pandas as pd
from datetime import date, timedelta
from typing import Iterator
import asyncio
from tqdm import tqdm
import pickle
import os
import dotenv
import matplotlib.pyplot as plt

In [5]:
dotenv.load_dotenv()
API_KEY = os.getenv("POLYGON_API_KEY")

In [6]:
constants.set_polygon_key(API_KEY)
pd.set_option("display.max_rows", None)
pd.set_option("display.max_columns", None)

Run the below cells to get the most recent financials data in SEC parser. Should be run every month or two.

In [7]:
processor.download_sec_data()

Data not stale. Skipping download. To override, set force_update=True or change max_stale_days.


### Data collection

In [8]:
async def gather_with_concurrency(n, *coros):
    semaphore = asyncio.Semaphore(n)
    async def sem_coro(coro):
        async with semaphore:
            return await coro
    return await asyncio.gather(*(sem_coro(c) for c in coros), return_exceptions=True)

In [9]:
def try_get_value(value_func):
    try:
        return value_func()
    except:
        return None

In [10]:
async def get_data(stocks: dict, query_date: str = None) -> dict:
    """
    Gather all the data needed for the embeddings for each given stock.

    Args:
        stocks (dict): Dictionary of stocks to gather data for: {ticker (str): stock (stock.Stock)}
        query_date (str, optional): Date in YYYY-MM-DD format. Defaults to None, which gets the latest metric.
    
    Returns:
        dict: Dictionary of data for each stock: {ticker (str): data (dict)}
    """

    if query_date is None:
        query_date = date.today().strftime("%Y-%m-%d")

    # get price and rsi for each stock
    price_coros, rsi_coros = [], []
    for ticker, s in stocks.items():
        price_coros.append(s.get_price(query_date))
        rsi_coros.append(s.get_rsi(query_date))
    results = await gather_with_concurrency(50, *price_coros, *rsi_coros)
    price_dict = {ticker: price for ticker, price in zip(stocks.keys(), results[: len(stocks)])}
    rsi_dict = {ticker: rsi for ticker, rsi in zip(stocks.keys(), results[len(stocks) :])}

    data = {}
    for ticker, s in stocks.items():
        try:
            price = price_dict[ticker]
            shares = s.get_shares_outstanding(query_date)
            market_cap = shares * price
        except:
            price = None
            market_cap = None

        # get price to book
        try:
            book_val = s.get_book_value(query_date)
            bvps = book_val / shares
            pb_ratio = price / bvps
        except:
            pb_ratio = None

        # get price to earnings
        try:
            net_income = s.get_net_income(query_date)
            try:
                preferred_div = s.get_preferred_dividends(query_date)
            except:
                preferred_div = 0
            eps = (net_income - preferred_div) / shares
            pe_ratio = price / eps
        except:
            pe_ratio = None

        # get price to sales
        try:
            revenue = s.get_revenue(query_date)
            sps = revenue / shares
            ps_ratio = price / sps
        except:
            ps_ratio = None

        # get price to cash flow
        try:
            ocf = s.get_operating_cash_flow(query_date)
            cfps = ocf / shares
            pcf_ratio = price / cfps
        except:
            pcf_ratio = None

        # get ev to ebitda
        try:
            debt = s.get_total_debt(query_date)
            cash = s.get_cash(query_date)
            ev = market_cap + debt - cash
            ebitda = s.get_ebitda(query_date)
            ev_ebitda = ev / ebitda
        except:
            ev_ebitda = None

        # get shareholder yield
        try:
            cash_dividends = s.get_cash_dividends(query_date)
        except:
            cash_dividends = 0
        try:
            share_repurchases = s.get_share_repurchases(query_date)
        except:
            share_repurchases = 0
        try:
            share_issuances = s.get_share_issuances(query_date)
        except:
            share_issuances = 0
        try:
            debt_paydown = s.get_debt_paydown(query_date)
        except:
            debt_paydown = 0
        try:
            debt_issuance = s.get_debt_issuance(query_date)
        except:
            debt_issuance = 0

        try:
            shareholder_yield = (
                cash_dividends
                + share_repurchases
                - share_issuances
                + debt_paydown
                - debt_issuance
            ) / market_cap
        except:
            shareholder_yield = None

        try:
            wacc = s.get_wacc(query_date)
        except Exception:
            wacc = None
            
        rsi = rsi_dict[ticker]
        industry = s.industry

        data[ticker] = {
            "pb_ratio": pb_ratio,
            "pe_ratio": pe_ratio,
            "ps_ratio": ps_ratio,
            "pcf_ratio": pcf_ratio,
            "ev_ebitda": ev_ebitda,
            "shareholder_yield": shareholder_yield,
            "rsi": rsi,
            "price": price,
            "industry": industry,
            "wacc": wacc,
        }

    return data

### Data Cleaning Logic

In [11]:
def clean_data(data: dict, universe: list) -> tuple[pd.DataFrame, float]:
    """
    Handle data corrections, removal of na values, etc..

    Args:
        data (dict): Dictionary of data for each stock: {ticker (str): data (dict)}
        universe (list): Set of tickers to rank (in case data contains more stocks than the universe).

    Returns:
        tuple[pd.DataFrame, float]: Tuple of the ranked DataFrame and the data coverage percentage.
    """
    
    df = pd.DataFrame(data).T
    available_tickers = df.index.intersection(universe)
    df = df.loc[available_tickers]

    # drop rows with nan values, determine coverage percentage
    df = df.dropna()
    coverage = len(df) / len(universe)

    # sort by rsi (high to low)
    df = df[df["rsi"].apply(lambda x: isinstance(x, float))]
    df = df.sort_values("rsi", ascending=False)

    return df, coverage

### Run Data Collection for Current Date

In [12]:
run_for_current = False

In [13]:
if run_for_current:
    stocks = {}
    failed = set()
    sp500 = lookups.get_sp500_tickers()
    for ticker in sp500:
        try:
            stocks[ticker] = stock.Stock(ticker)
        except:
            failed.add(ticker)
            continue

In [14]:
if run_for_current:
    vc2_data = await get_data(stocks)
    df, coverage = clean_data(vc2_data, sp500)
    print(coverage)
    print(df)

### Gather Historical Data

In [15]:
def daterange(start_date: str, end_date: str, interval: int = 1) -> Iterator[date]:
    """
    Generate a range of dates between start_date and end_date.
    
    Args:
        start_date (str): Start date in YYYY-MM-DD format.
        end_date (str): End date in YYYY-MM-DD format.
        interval (int, optional): Interval between dates in days. Defaults to 1.
    
    Returns:
        Iterator[str]: Iterator of dates in string format between start_date and end_date.
    """

    start_date = date.fromisoformat(start_date)
    end_date = date.fromisoformat(end_date)
    delta = end_date - start_date
    for i in range(0, delta.days + 1, interval):
        yield str(start_date + timedelta(days=i))

In [16]:
def get_sp500_universe(start_date: str, end_date: str, interval: int) -> set:
    # get all tickers in the S&P 500 between start_date and end_date
    all_sp500_tickers = set()
    for d in daterange(start_date, end_date, interval):
        all_sp500_tickers.update(lookups.get_sp500_tickers(d))

    return all_sp500_tickers

In [17]:
def process_and_get_stocks(start_date: str, end_date: str, interval: int) -> dict:
    # initialize stocks dictionary: {ticker: stock.Stock}
    stocks = {}
    for ticker in get_sp500_universe(start_date, end_date, interval):
        try:
            processor.process_sec_json(ticker)
            stocks[ticker] = stock.Stock(ticker)
        except Exception as e:
            print(f"Error getting stock data for {ticker}: {e}")

    return stocks

In [18]:
def industry_encoding(data: dict):
    """
    Encodes industry value as a one-hot encoding in dataset.

    Args:
        data (dict): Dictionary of data for each stock at each timepoint: {date (str) : {ticker (str): data (dict)}}

    Returns:
        dict: Dictionary of data for each stock at each timepoint with industry one-hot encoding: {date (str) : {ticker (str): data (dict)}}
    """

    industries = set(d["industry"] for date_data in data.values() for d in date_data.values())
    industry_dict = {industry: i for i, industry in enumerate(sorted(industries))}

    for date_data in data.values():
        for ticker, metrics in date_data.items():
            industry_encoding = [0] * len(industries)
            industry_encoding[industry_dict[metrics["industry"]]] = 1
            date_data[ticker]["industry_encoded"] = industry_encoding

    return data

In [19]:
async def gather_historical_data(stocks: dict, start_date: str, end_date: str, interval: int) -> dict:
    """
    Gather data for all stocks in the S&P 500 between start_date and end_date.

    Args:
        stocks (dict): Dictionary of stocks to gather data for: {ticker (str): stock (stock.Stock)}
        start_date (str): Start date of the backtest.
        end_date (str): End date of the backtest.
        interval (int): Interval between dates in days.

    Returns:
        dict: {str(date): {ticker: {data}}}
    """
    # get all data for all stocks in the S&P 500 between start_date and end_date
    # {str(date): {ticker: {data}}
    data = {}
    for d in tqdm(daterange(start_date, end_date, interval)):
        data[d] = await get_data(stocks, d)

    return data

### Historical Data Gathering Results

In [20]:
BACKTEST_DIR = "saved_data"

def save_backtest_data(data: dict, start_date: str, end_date: str, interval: int, save_dir: str = BACKTEST_DIR):
    os.makedirs(save_dir, exist_ok=True)
    with open(os.path.join(save_dir, f"{start_date}_{end_date}_{interval}.pickle"), "wb") as f:
        pickle.dump(data, f)

In [21]:
GATHER_START_DATE_6M = "2023-07-01"
GATHER_START_DATE_1Y = "2023-01-01"
GATHER_START_DATE_2Y = "2022-01-01"
GATHER_START_DATE_10Y = "2013-01-01"
GATHER_END_DATE = "2024-01-01"
GATHER_INTERVAL_MONTHLY, GATHER_INTERVAL_WEEKLY, GATHER_INTERVAL_DAILY = 30, 7, 1

In [22]:
stocks = process_and_get_stocks(GATHER_START_DATE_10Y, GATHER_END_DATE, 1)

Error getting stock data for RDC: 'RDC'
Error getting stock data for PEAK: 'PEAK'


KeyboardInterrupt: 

In [ ]:
# Get monthly backtest data
monthly = await gather_historical_data(stocks, GATHER_START_DATE_10Y, GATHER_END_DATE, GATHER_INTERVAL_MONTHLY)
save_backtest_data(monthly, GATHER_START_DATE_10Y, GATHER_END_DATE, GATHER_INTERVAL_MONTHLY)

0it [00:00, ?it/s]C:\Users\dylan\AppData\Local\Temp\ipykernel_11352\924566295.py:38: RuntimeWarning: divide by zero encountered in scalar divide
  bvps = book_val / shares
C:\Users\dylan\AppData\Local\Temp\ipykernel_11352\924566295.py:50: RuntimeWarning: divide by zero encountered in scalar divide
  eps = (net_income - preferred_div) / shares
C:\Users\dylan\AppData\Local\Temp\ipykernel_11352\924566295.py:58: RuntimeWarning: divide by zero encountered in scalar divide
  sps = revenue / shares
C:\Users\dylan\AppData\Local\Temp\ipykernel_11352\924566295.py:66: RuntimeWarning: divide by zero encountered in scalar divide
  cfps = ocf / shares
C:\Users\dylan\AppData\Local\Temp\ipykernel_11352\924566295.py:104: RuntimeWarning: divide by zero encountered in scalar divide
  shareholder_yield = (
C:\Users\dylan\AppData\Local\Temp\ipykernel_11352\924566295.py:67: RuntimeWarning: divide by zero encountered in scalar divide
  pcf_ratio = price / cfps
C:\Users\dylan\AppData\Local\Temp\ipykernel_1135

In [ ]:
# get weekly backtest data
weekly = await gather_historical_data(stocks, GATHER_START_DATE_1Y, GATHER_END_DATE, GATHER_INTERVAL_WEEKLY)
save_backtest_data(weekly, GATHER_START_DATE_1Y, GATHER_END_DATE, GATHER_INTERVAL_WEEKLY)

0it [00:00, ?it/s]C:\Users\dylan\AppData\Local\Temp\ipykernel_11352\924566295.py:38: RuntimeWarning: divide by zero encountered in scalar divide
  bvps = book_val / shares
C:\Users\dylan\AppData\Local\Temp\ipykernel_11352\924566295.py:50: RuntimeWarning: divide by zero encountered in scalar divide
  eps = (net_income - preferred_div) / shares
C:\Users\dylan\AppData\Local\Temp\ipykernel_11352\924566295.py:58: RuntimeWarning: divide by zero encountered in scalar divide
  sps = revenue / shares
C:\Users\dylan\AppData\Local\Temp\ipykernel_11352\924566295.py:66: RuntimeWarning: divide by zero encountered in scalar divide
  cfps = ocf / shares
C:\Users\dylan\AppData\Local\Temp\ipykernel_11352\924566295.py:104: RuntimeWarning: divide by zero encountered in scalar divide
  shareholder_yield = (
C:\Users\dylan\AppData\Local\Temp\ipykernel_11352\924566295.py:59: RuntimeWarning: divide by zero encountered in scalar divide
  ps_ratio = price / sps
1it [00:25, 25.52s/it]C:\Users\dylan\AppData\Local

In [ ]:
# get daily backtest data
daily = await gather_historical_data(stocks, GATHER_START_DATE_6M, GATHER_END_DATE, GATHER_INTERVAL_DAILY)
save_backtest_data(daily, GATHER_START_DATE_6M, GATHER_END_DATE, GATHER_INTERVAL_DAILY)

0it [00:00, ?it/s]C:\Users\dylan\AppData\Local\Temp\ipykernel_11352\924566295.py:59: RuntimeWarning: divide by zero encountered in scalar divide
  ps_ratio = price / sps
1it [00:23, 23.33s/it]C:\Users\dylan\AppData\Local\Temp\ipykernel_11352\924566295.py:59: RuntimeWarning: divide by zero encountered in scalar divide
  ps_ratio = price / sps
2it [00:48, 24.25s/it]C:\Users\dylan\AppData\Local\Temp\ipykernel_11352\924566295.py:59: RuntimeWarning: divide by zero encountered in scalar divide
  ps_ratio = price / sps
3it [01:09, 23.08s/it]C:\Users\dylan\AppData\Local\Temp\ipykernel_11352\924566295.py:59: RuntimeWarning: divide by zero encountered in scalar divide
  ps_ratio = price / sps
4it [01:33, 23.47s/it]C:\Users\dylan\AppData\Local\Temp\ipykernel_11352\924566295.py:59: RuntimeWarning: divide by zero encountered in scalar divide
  ps_ratio = price / sps
5it [01:54, 22.55s/it]C:\Users\dylan\AppData\Local\Temp\ipykernel_11352\924566295.py:59: RuntimeWarning: divide by zero encountered in

In [ ]:
# Get daily backtest data for 2Y
monthly = await gather_historical_data(stocks, GATHER_START_DATE_2Y, GATHER_END_DATE, GATHER_INTERVAL_DAILY)
save_backtest_data(monthly, GATHER_START_DATE_2Y, GATHER_END_DATE, GATHER_INTERVAL_DAILY)

0it [00:00, ?it/s]

C:\Users\dylan\AppData\Local\Temp\ipykernel_11352\924566295.py:67: RuntimeWarning: divide by zero encountered in scalar divide
  pcf_ratio = price / cfps
C:\Users\dylan\AppData\Local\Temp\ipykernel_11352\924566295.py:59: RuntimeWarning: divide by zero encountered in scalar divide
  ps_ratio = price / sps
1it [00:31, 31.64s/it]

In [23]:
def load_backtest_data(start_date: str, end_date: str, interval: int, save_dir: str = BACKTEST_DIR):
    with open(os.path.join(save_dir, f"{start_date}_{end_date}_{interval}.pickle"), "rb") as f:
        return pickle.load(f)

In [28]:
weekly = load_backtest_data(GATHER_START_DATE_10Y, GATHER_END_DATE, GATHER_INTERVAL_WEEKLY)
daily = load_backtest_data(GATHER_START_DATE_2Y, GATHER_END_DATE, GATHER_INTERVAL_DAILY)

# Combine results into one dataset, where dataset is {date (str) : {ticker (str): data (dict)}}
dataset = {**weekly, **daily}
dataset = industry_encoding(dataset)

df = pd.DataFrame.from_dict(
    {(i, j): dataset[i][j] for i in dataset.keys() for j in dataset[i].keys()},
    orient="index",
)

df.index.names = ["Date", "Ticker"]
df.dropna(inplace=True)

In [29]:
df.to_csv(f"{BACKTEST_DIR}/dataset.csv")